In [ ]:
from deepface import DeepFace
import pandas as pd
import os


os.chdir('../../rfiw2021')

In [3]:
df = pd.read_csv('Track1/sample0/val.txt', sep=" ", header=None)
df.columns = ['index', 'face1_path', 'face2_path', 'kinship', 'label']
df.drop('index', axis=1, inplace=True) 

In [112]:
from tqdm import tqdm

def analyze_batch(df, b, name):
    base_file = 'Track1/'
    
    age = []
    gender = []
    race = []

    for i, data in tqdm(df.iterrows(), total=len(df), desc=f"Processing batch {b}"):

        obj = DeepFace.analyze(img_path = base_file+data['face'], enforce_detection=False, silent=True,
            actions = ['age', 'gender', 'race']
        )[0]

        age.append(obj['age'])
        gender.append(obj['dominant_gender'])
        race.append(obj['dominant_race'])


    df['age'] = age
    df['gender'] = gender
    df['race'] = race

    df.to_csv(f'fiw/{name}-{b}.csv', index=False)

In [115]:
batch_size = 128

num_batches = len(df) // batch_size + 1 if len(df) % batch_size != 0 else len(df) // batch_size

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = (i + 1) * batch_size
    batch_df = df.iloc[start_idx:end_idx]

    if os.path.exists(f'fiw/test-{i}.csv'):
        continue

    analyze_batch(batch_df, i, 'test') 

Processing batch 40: 100%|██████████| 106/106 [00:48<00:00,  2.17it/s]


In [117]:
dfs = []
for i in range(40):
    dfs.append(pd.read_csv(f'fiw/test-{i}.csv'))

df = pd.concat(dfs).to_csv('fiw/test_with_deepface.csv', index=False)